In [1]:
from datetime import date, timedelta
import calendar 
import os
import glob

In [2]:
import pandas as pd
names = ["b1", "b2", "m1", "m2", "s1", "s2", "s3", "s4", "s5"]
inventory = []
sell = []
supply = []
for file in glob.glob("*-inventory.csv"):
    inventory.append(file)
    inventory.sort(key = str.lower)
for file in glob.glob("*-sell.csv"):
    sell.append(file)
    sell.sort(key = str.lower)
for file in glob.glob("*-supply.csv"):
    supply.append(file)
    supply.sort(key = str.lower)



by_year_dict = {
    '2006' : {'ap_sold' : 0, 'ap_stolen': 0, 'pen_sold': 0, 'pen_stolen': 0},
    '2007' : {'ap_sold' : 0, 'ap_stolen': 0, 'pen_sold': 0, 'pen_stolen': 0},
    '2008' : {'ap_sold' : 0, 'ap_stolen': 0, 'pen_sold': 0, 'pen_stolen': 0},
    '2009' : {'ap_sold' : 0, 'ap_stolen': 0, 'pen_sold': 0, 'pen_stolen': 0},
    '2010' : {'ap_sold' : 0, 'ap_stolen': 0, 'pen_sold': 0, 'pen_stolen': 0},
    '2011' : {'ap_sold' : 0, 'ap_stolen': 0, 'pen_sold': 0, 'pen_stolen': 0},
    '2012' : {'ap_sold' : 0, 'ap_stolen': 0, 'pen_sold': 0, 'pen_stolen': 0},
    '2013' : {'ap_sold' : 0, 'ap_stolen': 0, 'pen_sold': 0, 'pen_stolen': 0},
    '2014' : {'ap_sold' : 0, 'ap_stolen': 0, 'pen_sold': 0, 'pen_stolen': 0},
    '2015' : {'ap_sold' : 0, 'ap_stolen': 0, 'pen_sold': 0, 'pen_stolen': 0},
}

for j in range(len(inventory)):
    b1_inventory = pd.read_csv(inventory[j])
    b1_sell = pd.read_csv(sell[j])
    b1_supply = pd.read_csv(supply[j])
    
    
    date1 = date(2006,1,1)
    date2 = date(2015,12,31)
    delta = date2 - date1
    lst = []
    lst = [str(date1 + timedelta(i)) for i in range(delta.days + 1)]
    

    b1_inv = b1_inventory.to_dict()
    b1_sup = b1_supply.to_dict()
    b1_s = b1_sell.to_dict()
    
    
    apple_sold = 0
    pen_sold = 0
    apple_cur = 0
    pen_cur = 0
    apple_day = []
    pen_day = []
    apple_stolen = []
    pen_stolen = []
    stolen_days = []
    cur_date = '2006-01-01'
    cur_date_changed = True
    sup_index = 0
    inv_index = 0
    
    for i in range(len(b1_s['sku_num'])):
        if cur_date != b1_s['date'][i]:
            apple_cur -= apple_sold
            pen_cur -= pen_sold
            if int(cur_date.split('-')[2]) == calendar.monthrange(int(cur_date.split('-')[0]),int(cur_date.split('-')[1]))[1]:
                ap_steal = apple_cur - b1_inv['apple'][inv_index]
                pe_steal = pen_cur - b1_inv['pen'][inv_index]
                apple_cur = b1_inv['apple'][inv_index]
                pen_cur = b1_inv['pen'][inv_index]
                inv_index += 1
                apple_stolen.append(ap_steal)
                pen_stolen.append(pe_steal)
                stolen_days.append(cur_date)
                by_year_dict[cur_date.split('-')[0]]['ap_stolen'] += ap_steal
                by_year_dict[cur_date.split('-')[0]]['pen_stolen'] += pe_steal
            cur_date = b1_s['date'][i]
            cur_date_changed = True
            apple_day.append(apple_cur)
            pen_day.append(pen_cur)
            apple_sold = 0
            pen_sold = 0
        if cur_date_changed:
            if cur_date.split('-')[2] == '01'or cur_date.split('-')[2] == '15':
                apple_cur += b1_sup['apple'][sup_index]
                pen_cur += b1_sup['pen'][sup_index]
                sup_index += 1
            cur_date_changed = False
        if b1_s['sku_num'][i].split('-')[2] == 'ap':
            apple_sold += 1
            by_year_dict[cur_date.split('-')[0]]['ap_sold'] += 1
        if b1_s['sku_num'][i].split('-')[2] == 'pe':
            pen_sold += 1
            by_year_dict[cur_date.split('-')[0]]['pen_sold'] += 1
    apple_cur -= apple_sold
    pen_cur -= pen_sold
    if int(cur_date.split('-')[2]) == calendar.monthrange(int(cur_date.split('-')[0]),int(cur_date.split('-')[1]))[1]:
                ap_steal = apple_cur - b1_inv['apple'][inv_index]
                pe_steal = pen_cur - b1_inv['pen'][inv_index]
                apple_cur = b1_inv['apple'][inv_index]
                pen_cur = b1_inv['pen'][inv_index]
                apple_stolen.append(ap_steal)
                pen_stolen.append(pe_steal)
    apple_day.append(apple_cur)
    pen_day.append(pen_cur)
    stolen_days.append(cur_date)
    
    dct = {
        'date': lst,
        'apple': apple_day,
        'pen': pen_day
    }
    
    first_ex = pd.DataFrame(dct)
    name1 = "out_input_MS-"+names[j]+"-daily"
    first_ex.to_csv(name1, sep='\t', encoding='utf-8')
    
    dict_of_stolen = {
        'date': stolen_days,
        'apple': apple_stolen,
        'pen': pen_stolen
    }
    
    second_ex = pd.DataFrame(dict_of_stolen)
    name2 = "out_input_MS-"+names[j]+"-steal"
    second_ex.to_csv(name2, sep='\t', encoding='utf-8')    




In [34]:
years = []
states = []
lst_ap_sold = []
lst_ap_stolen = []
lst_pen_sold = []
lst_pen_stolen = []

for key in by_year_dict.keys():
    years.append(key)
    states.append('MS')
for dct in by_year_dict:
    temp = by_year_dict.get(dct)
    for key in temp:
        if key == 'ap_sold':
            lst_ap_sold.append(temp.get(key)) 
        if key == 'ap_stolen':
            lst_ap_stolen.append(temp.get(key))
        if key == 'pen_sold':
            lst_pen_sold.append(temp.get(key))
        if key == 'pen_stolen':
            lst_pen_stolen.append(temp.get(key))

third_ex_dict = {
    'year': years,
    'state': states,
    'apple_sold': lst_ap_sold,
    'apple_stolen': lst_ap_stolen,
    'pen_sold': lst_pen_sold,
    'pen_stolen': lst_pen_stolen
}

In [35]:
third_ex = pd.DataFrame(third_ex_dict)
third_ex

,year,state,apple_sold,apple_stolen,pen_sold,pen_stolen
0,2006,MS,2152006,418,155633,461
1,2007,MS,2150384,377,154730,346
2,2008,MS,2163559,383,154597,382
3,2009,MS,2152502,433,155409,454
4,2010,MS,2149787,418,155523,441
5,2011,MS,2154860,436,154158,452
6,2012,MS,2160040,381,155798,421
7,2013,MS,2157901,361,154496,444
8,2014,MS,2153434,433,154687,441
9,2015,MS,2152497,344,153562,361
